<a href="https://colab.research.google.com/github/brucewayneee1/ML-echo/blob/master/CodeBase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Using CUDA
use_gpu = torch.cuda.is_available()
if use_gpu:
    print("Using CUDA")
else: print("Using CPU")


Using CPU


In [ ]:
!git clone https://github.com/brucewayneee1/ML-echo

fatal: destination path 'ML-echo' already exists and is not an empty directory.


In [ ]:
TrainTest = namedtuple('TrainTest', ['train', 'test'])

def get_classes():
  classes = ['2C', '3C', '4C']
  return classes

def prepare_data():
  transform_train = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomCrop((224, 224)),
    transforms.RandomHorizontalFlip(),                                
    transforms.ToTensor(),
  ])
  transform_test = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.CenterCrop((224, 224)),
    transforms.ToTensor(),
  ])
  trainset = torchvision.datasets.ImageFolder(root="/content/ML-echo/DATA_CHAMBER_2021/train", transform=transform_train)
  testset = torchvision.datasets.ImageFolder(root="/content/ML-echo/DATA_CHAMBER_2021/test", transform=transform_test)
  return TrainTest(train=trainset, test=testset)

def prepare_loader(datasets):
  batch = 32
  worker = 4
  trainloader = DataLoader(dataset=datasets.train, batch_size=batch, shuffle=True, num_workers=worker)
  testloader = DataLoader(dataset=datasets.test, batch_size=batch, shuffle=False, num_workers=worker)
  return TrainTest(train=trainloader, test=testloader)

In [ ]:
def train_epoch(epoch, model, loader, loss_func, optimizer, device):
  model.train()
  running_loss = 0.0
  reporting_steps = 40
  step = 0
  for images, labels in loader:
    step += 1
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    loss = loss_func(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if step % reporting_steps == reporting_steps - 1:
      print(f"Epoch {epoch} step {step} ave_loss {running_loss/reporting_steps:.4f}")
      running_loss = 0.0

def test_epoch(epoch, model, loader, device):
  ytrue = []
  ypred = []
  with torch.no_grad():
    model.eval()
    for images, labels in loader:
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs, dim=1)
      ytrue += list(labels.cpu().numpy())
      ypred += list(predicted.cpu().numpy())

  return ypred, ytrue

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable

import cv2
import numpy as np
import matplotlib.pyplot as plt


import torchvision
from torchvision import models,transforms
from torchvision.datasets import ImageFolder

from collections import namedtuple

from sklearn.metrics import classification_report

import time
import os
import copy


In [ ]:
def main(model='vgg16'):
  classes = get_classes()
  datasets = prepare_data()
  loaders = prepare_loader(datasets)
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print("Num Images in train set:", len(datasets.train))
  print("Num Images in test set:", len(datasets.test))
  print("Num batch in train set: ", len(loaders.train))
  print("class: ", datasets.train.class_to_idx)
  print("image size: ", datasets.train[0][0].shape)
  print("device:", device)
  print("model:", model)

  if model == 'vgg16':
    PATH='./vgg16.pth'  
    model = model = torchvision.models.vgg16()
    model.classifier[6] = nn.Linear(in_features=4096, out_features=3)
  elif model == 'vgg19':
    PATH='./vgg19.pth'
    model = torchvision.models.vgg16()
    model.classifier[6] = nn.Linear(in_features=4096, out_features=3)
  elif model == 'resnet50':
    PATH='./resnet50.pth'
    model = torchvision.models.vgg16()
    model.classifier[6] = nn.Linear(in_features=2048, out_features=3) 
  
  model.to(device)
  loss_func = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

  accuracies = []
  for epoch in range(10):
    print("---------------------------------------------------------------")
    print(f"\nEpoch {epoch} report: ")
    train_epoch(epoch, model, loaders.train, loss_func, optimizer, device)
    ypred_test, ytrue_test = test_epoch(epoch, model, loaders.test, device)
    print("Test report: \n", classification_report(ytrue_test, ypred_test, target_names=classes))

  return model

In [ ]:
result_VGG16 = main(model='vgg16')

Num Images in train set: 6717
Num Images in test set: 1607
Num batch in train set:  210
class:  {'2C': 0, '3C': 1, '4C': 2}
image size:  torch.Size([3, 224, 224])
device: cpu
model: vgg16


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


NameError: ignored

In [ ]:
result_VGG19 = run(model='vgg19')

In [ ]:
result_Resnet50 = run(model='resnet50')